# Student Id :ChinmayNandkishor_Mathakari@student.uml.edu(02007302) 

# Import libraries

In [10]:


import re
import os
import sys
import html
import spacy
import pandas as pd
import matplotlib.pyplot as plt


from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

ImportError: cannot import name '_OneToOneFeatureMixin' from 'sklearn.base' (/Users/sagar/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py)

# Q1.) Clean and remove short bug reports as in TextClassification.ipynb. Organize the remaining data to a data frame called df.

In [ ]:
file = "eclipse_jdt.csv"
df = pd.read_csv(file)
df = df[['Issue_id','Priority','Component','Title','Description']]
df.head(5)

In [ ]:
df = df[['Title','Description','Priority']]
df = df.dropna()
df['text'] = df['Title'] + ' ' + df['Description']
df = df.drop(columns=['Title','Description'])
df.head(5)

In [ ]:
# Method defined to clean text 
def clean(text):
    text = html.unescape(text) # convert html escapes like &amp; to characters.
    text = re.sub(r'<[^<>]*>', ' ', text) # tags like <tab>
    text = re.sub(r'\[([^\[\]]*)\]\([^\(\)]*\)', r'\1', text) # markdown URLs like [Some text](https://....)
    text = re.sub(r'\[[^\[\]]*\]', ' ', text) # text or code in brackets like [0]
    text = re.sub(r'(?:^|\s)[&#<>{}\[\]+|\\:-]{1,}(?:\s|$)', ' ', text) # standalone sequences of specials, matches &# but not #cool
    text = re.sub(r'(?:^|\s)[\-=\+]{2,}(?:\s|$)', ' ', text) # standalone sequences of hyphens like --- or ==
    text = re.sub('[^A-Za-z]+', ' ', text) # remove any characters other than alphabets
    text = re.sub(r'\s+', ' ', text) # sequences of white spaces
    return text.strip()


df['text'] = df['text'].apply(clean)
df = df[df['text'].str.len() > 50]
df.head(5)

# Q2.) Generate a balanced data frame called df_balanced from df by restricting the number of entries in the P3 category to 5000.

In [ ]:
# Filter bug reports with priority P3 and sample 5000 rows from it
df_sample_P3 = df[df['Priority'] == 'P3'].sample(n = 5000, random_state=123)

# Create a separate dataframe containing all other bug reports
df_sample_Rest = df[df['Priority'] != 'P3']

# Concatenate the two dataframes to create the new balanced bug reports dataset
df_balanced = pd.concat([df_sample_Rest, df_sample_P3])

# Check the status of the class imbalance
df_balanced['Priority'].value_counts()
df_balanced

# Q3.) Lemmatize words and use only 'nav' lemmas for classifications, where 'nav' stands for nouns, pronouns, adjectives, adverbs, and verbs as in FeatureExtraction.

In [ ]:
#logic for the feature extraction such as nouns, pronouns, adjectives, adverbs and verbs

nlp = spacy.load("en_core_web_sm")
token = list()
speech = str(df['text'])
doc = nlp(speech)

nav = ("NOUN", "VERB","ADJ","ADV")

count = 0


print("Feature Extraction")
print("--------------------------------------------------------------------------------------------------------------------")
for sent in doc.sents:
    for i in sent:
        if i.pos_ in nav :
            word = re.sub(r"[,.“”]", "", str(i.lemma_))
            word = word.split("\\n")
            print(f"Words :   {word}")
            token.append(word[0])
            count+=1
            
print("\nTotal Count : " , count)

In [ ]:
df_new = df_balanced[['text', 'Priority']]
df_new = df_new.dropna()
print(df_new)

In [ ]:
'''# Loading the balanced dataframe

df_new = df_balanced[['text', 'Priority']]
df_new = df_new.dropna()

# Step 1 - Data Preparation

df_new['text'] = df_new['text'].apply(clean)

# Step 2 - Train-Test Split
X_train_new, X_test_new, Y_train_new, Y_test_new = train_test_split(df_new['text'],
                                                    df_new['Priority'],
                                                    test_size=0.2,
                                                    random_state=42,
                                                    stratify=df_new['Priority'])

print('Number of Training Data : ', X_train_new.shape[0])
print('Number of Test Data : ', X_test_new.shape[0])
'''

In [ ]:
'''# Step 3 - Training the Machine Learning model

tfidf_new = TfidfVectorizer(min_df=10, ngram_range=(1, 2), stop_words="english")
X_train_new_tf = tfidf_new.fit_transform(X_train_new)

model_LSVC_new = LinearSVC(random_state=0, tol=1e-5) ### using LinearSVC
model_LSVC_new.fit(X_train_new_tf, Y_train_new)
'''

In [ ]:
'''# Step 4 - Model Evaluation

X_test_new_tf = tfidf_new.transform(X_test_new)
Y_pred_LSVC_new = model_LSVC_new.predict(X_test_new_tf)'''

In [ ]:
'''# Vectorization

#tfidf = TfidfVectorizer(min_df = 10, ngram_range=(1,2), stop_words="english")
df_new_tf = tfidf_new.fit_transform(df_new['text']).toarray()

# Cross Validation with 5 batches (folds)

scores = cross_val_score(estimator = model_LSVC_new,
                         X = df_new_tf,
                         y = df_new['Priority'],
                         cv = 5) 

print ("Validation scores for each iteration : ", scores)
print ("Mean value of validation scores : ", scores.mean())
print ("Standard deviation of validation scores : ", scores.std())'''

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.dummy import DummyClassifier


training_pipeline = Pipeline(
    steps=[('tfidf', TfidfVectorizer(stop_words="english")), 
           ('LinearSVC', LinearSVC(random_state=21, tol=1e-5))])

grid_param = [{
'tfidf__min_df': [5, 10],
'tfidf__ngram_range': [(1, 3), (1, 6)],
'LinearSVC__penalty': ['l2'],
'LinearSVC__loss': ['hinge'],
'LinearSVC__max_iter': [10000]
}, {
'tfidf__min_df': [5, 10],
'tfidf__ngram_range': [(1, 3), (1, 6)],
'LinearSVC__C': [1, 10],
'LinearSVC__tol': [1e-2, 1e-3]
}]


gridSearchProcessor = GridSearchCV(estimator=training_pipeline,
                                   param_grid=grid_param,
                                   cv=5)

gridSearchProcessor.fit(df_balanced['text'], df_balanced['Priority'])


print("\n")
print("Grid Search Results : ")
print("------------------------------------------------------------------------------")
best_params = gridSearchProcessor.best_params_
print("Best_Parameters : ", best_params)

best_result = gridSearchProcessor.best_score_
print("\n")
print("Best_Result : ", best_result)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier

training_pipeline = Pipeline(
    steps=[('tfidf', TfidfVectorizer(stop_words="english")), 
           ('SVC', SVC(random_state=21, tol=1e-5))])

tuned_parameters = [{
    'tfidf__min_df': [5, 10],
    'tfidf__ngram_range': [(1, 3), (1, 6)],
    'SVC__C': [1, 10, 100, 1000],
    'SVC__kernel': ['rbf', 'linear'], 
    'SVC__gamma': [1e-3, 1e-4],
    "SVC__tol": [1e-2, 1e-10]
}]

gridSearchProcessor = GridSearchCV(estimator=training_pipeline,
                                   param_grid=tuned_parameters,
                                   cv=2, n_jobs=-1)

gridSearchProcessor.fit(df_balanced['text'], df_balanced['Priority'])
print()
print("Grid Search Results : ")
print("------------------------------------------------------------------------------")
best_params_svc = gridSearchProcessor.best_params_
print("Best_Parameters : ", best_params_svc)

best_result_svc = gridSearchProcessor.best_score_
print("Best_Result : ", best_result_svc)

In [ ]:
# Loading the balanced dataframe

df_new = df_balanced[['text', 'Priority']]
df_new = df_new.dropna()

# Step 1 - Data Preparation

df_new['text'] = df_new['text'].apply(clean)

# Step 2 - Train-Test Split
X_train, X_test, Y_train, Y_test = train_test_split(df_new['text'],
                                                    df_new['Priority'],
                                                    test_size=0.2, ### 80-20 train-test split
                                                    random_state=42,
                                                    stratify=df_new['Priority'])

print('Number of Training Data : ', X_train.shape[0])
print('Number of Test Data : ', X_test.shape[0])

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(min_df = 10, ngram_range=(1,2), stop_words="english") 
### from sklearn
### min_df = 10 means that terms appearing in strickly less than 10 documents are ignored
X_train_tf = tfidf.fit_transform(X_train)

model_LSVC = LinearSVC(random_state=0, tol=1e-5) 
### tol: Tolerance for stopping criteria; 1e-5 = e^{-5} = exp(-5)
model_LSVC.fit(X_train_tf, Y_train)

X_test_tf = tfidf.transform(X_test)
Y_pred_LSVC = model_LSVC.predict(X_test_tf) ### evaluation
print("\n")
print ('Accuracy_Score : ', accuracy_score(Y_test, Y_pred_LSVC))

In [ ]:
from sklearn.datasets import load_iris
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

tfidf = TfidfVectorizer(min_df = 10, ngram_range=(1,2), stop_words="english") 
### from sklearn
### min_df = 10 means that terms appearing in strickly less than 10 documents are ignored
X_train_tf = tfidf.fit_transform(X_train)

model_SVC = SVC(random_state=0, tol=1e-5) 
### tol: Tolerance for stopping criteria; 1e-5 = e^{-5} = exp(-5)
model_SVC.fit(X_train_tf, Y_train)

X_test_tf = tfidf.transform(X_test)
Y_pred_SVC = model_SVC.predict(X_test_tf) ### evaluation
print("\n")
print ('Accuracy_Score : ', accuracy_score(Y_test, Y_pred_SVC))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

LSVC_tfidf = TfidfVectorizer(min_df=best_params['tfidf__min_df'],
                           ngram_range=best_params['tfidf__ngram_range'],
                           stop_words='english')
LSVC_X_train_tf = LSVC_tfidf.fit_transform(X_train)


best_modal_lsvc=LinearSVC(random_state=21,penalty = 'l2',max_iter = 10000,
                          loss= 'hinge')
best_modal_lsvc.fit(LSVC_X_train_tf,Y_train)





SVC_tfidf = TfidfVectorizer(min_df = best_params['tfidf__min_df'],
                           ngram_range = best_params['tfidf__ngram_range'],
                           stop_words = 'english')

SVC_X_train_tf = SVC_tfidf.fit_transform(X_train)
best_modal_svc = SVC(random_state=21)
best_modal_svc.fit(SVC_X_train_tf,Y_train)

In [ ]:
LSVC_X_test_tf=LSVC_tfidf.transform(X_test)
Y_pred_LSVC=best_modal_lsvc.predict(LSVC_X_test_tf)
print ('\nLinearSVC_Accuracy_Score : ', accuracy_score(Y_test, Y_pred_LSVC))
print("-------------------------------------------------------------------------------------------------------")
print (classification_report(Y_test, Y_pred_LSVC))



SVC_X_test_tf=SVC_tfidf.transform(X_test)
Y_pred_SVC=best_modal_svc.predict(SVC_X_test_tf)
print ('\nSVC_Accuracy_Score - ', accuracy_score(Y_test, Y_pred_SVC))
print("-------------------------------------------------------------------------------------------------------")
print (classification_report(Y_test, Y_pred_SVC))

In [ ]:
linear_modal_SVC = SVC(C = best_params_svc['SVC__C'],
                       kernel = 'linear',gamma=best_params_svc['SVC__gamma'],tol=best_params_svc['SVC__tol'],probability=True,random_state=42)

linear_modal_SVC.fit(SVC_X_train_tf, Y_train)

SVC_X_test_tf = SVC_tfidf.transform(X_test)

Y_pred_Linear_Kernal_SVC= linear_modal_SVC.predict(SVC_X_test_tf)
print()
print ('\nAccuracy Score - ', accuracy_score(Y_test, Y_pred_Linear_Kernal_SVC))
print (classification_report(Y_test, Y_pred_Linear_Kernal_SVC))

In [ ]:
rbf_modal_SVC=SVC(C = best_params_svc['SVC__C'],kernel = 'rbf',gamma=best_params_svc['SVC__gamma'],tol=best_params_svc['SVC__tol'],probability=True,random_state=42)
rbf_modal_SVC.fit(SVC_X_train_tf, Y_train)

SVC_X_test_tf=SVC_tfidf.transform(X_test)

Y_pred_rbf_Kernal_SVC= rbf_modal_SVC.predict(SVC_X_test_tf)
print()
print ('\nAccuracy Score - ', accuracy_score(Y_test, Y_pred_rbf_Kernal_SVC))
print (classification_report(Y_test, Y_pred_rbf_Kernal_SVC))

In [ ]:
Y_pred_LSVC=best_modal_lsvc.predict(LSVC_X_test_tf)
LSVC_results= pd.DataFrame({'text':X_test.values,'actual':Y_test.values,'predicted':Y_pred_LSVC})
print(LSVC_results)

Y_pred_SVC=model_SVC.predict(X_test_tf)
SVC_results=pd.DataFrame({'text':X_test.values,'actual':Y_test.values,'predicted':Y_pred_SVC})
print(SVC_results)

Y_pred_Linear_Kernal_SVC=linear_modal_SVC.predict(SVC_X_test_tf)
Linear_SVC_results=pd.DataFrame({'text':X_test.values,'actual':Y_test.values,'predicted':Y_pred_Linear_Kernal_SVC})

Y_pred_rbf_Kernal_SVC=rbf_modal_SVC.predict(SVC_X_test_tf)
rbf_SVC_results=pd.DataFrame({'text':X_test.values,'actual':Y_test.values,'predicted':Y_pred_rbf_Kernal_SVC})

In [ ]:
linear_modal_SVC.coef_

# coef = linear_modal_SVC.coef_[8].A[0]
# vocabulary_positions = coef.argsort()
# vocabulary = tfidf.get_feature_names()

# top_words = 5
# top_positive_coef = vocabulary_positions[-top_words:].tolist()
# top_negative_coef = vocabulary_positions[:top_words].tolist()
# print(top_positive_coef)
# print(top_negative_coef)

# core_ui = pd.DataFrame([[vocabulary[c], coef[c]] for c in top_positive_coef + top_negative_coef], 
#                           columns=["feature", "coefficient"]).sort_values("coefficient")

coef = linear_modal_SVC.coef_[9].A[0]
print(coef)
vocabulary_positions = coef.argsort()
vocabulary = tfidf.get_feature_names()
top_words = 5
top_positive_coef = vocabulary_positions[-top_words:].tolist()
top_negative_coef = vocabulary_positions[:top_words].tolist()
print(top_positive_coef)
print(top_negative_coef)
print(top_positive_coef + top_negative_coef)
core_ui = pd.DataFrame([[vocabulary[c], coef[c]] for c in range(len(top_positive_coef+top_negative_coef))], 
                          columns=["feature", "coefficient"]).sort_values("coefficient")
core_ui.set_index("feature")

In [ ]:
from sklearn.pipeline import make_pipeline
pipeline = make_pipeline(tfidf, rbf_modal_SVC)
result = pd.DataFrame({ 'text': X_test.values, 'actual': Y_test.values, 'predicted': Y_pred_SVC })
class_names = ["text", "predicted"]
prob = rbf_modal_SVC.predict_proba(SVC_X_test_tf)
prob_svc = prob
# new dataframe for explainable results
er = result.copy().reset_index() ### er stands for "explain result"
explain_result = er
er['max_probability'] = er[class_names].max(axis=1)
class_names = ["text", "predict"]
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=class_names)
er[er["predicted"] != er["actual"]].head(10)